# <centre>WORKFLOW</centre>
    

## Generate initial configuration

In [ ]:
!gmx insert-molecules -ci LNPN.gro -box 15 15 15 -nmol 200 -radius 0.21 -try 1000 -o LNPN-final.gro 

In [ ]:
!gmx insert-molecules -f LNPN-final.gro -ci RNA-center.gro -box 17 17 17 -nmol 1 -radius 0.21 -try 1000 -o LNPC-final.gro 

In [ ]:
!gmx insert-molecules -f LNPN-final.gro -ci DPPC.gro -box 15 15 15 -nmol 40 -radius 0.21 -try 500 -o DPPC-final.gro 

In [ ]:
!gmx insert-molecules -f DPPC-final.gro -ci CHOL.gro -box 15 15 15 -nmol 160 -radius 0.21 -try 1000 -o system.gro 

## Solvate and Ionize

In [ ]:
!gmx solvate -cp system.gro -cs water.gro -p lnpn-dipc-chol.top -box 15 15 15 -radius 0.21 -o system-sol.gro

### Run script to replace water molecules with antifreeze beads

In [ ]:
!python temp.py

In [ ]:
!{"gmx genion -s system-sol-min.tpr -p lnpn-dipc-chol.top -pname NA+ -nname CL- -neutral -o system-sol-ion.gro <<EOF\n5\\nEOF"}

## Minimization

In [ ]:
!gmx grompp -f em.mdp -c system-sol-ion.gro -p lnpn-dipc-chol.top -o system-sol-ion.tpr -maxwarn 1

In [ ]:
!gmx mdrun -deffnm system-ion-min -s system-sol-ion.tpr -v -c system-ion-min.gro

## Equilibriation

In [ ]:
!{"gmx grompp -f nvt.mdp -c system-sol-min.gro -n index.ndx -r system-sol.gro -p lnpn-dipc-chol.top -o nvt.tpr"}

In [ ]:
!{"gmx mdrun -v -deffnm nvt"}

In [ ]:
!{"gmx grompp -f npt.mdp -n index.ndx -c nvt.gro -r nvt.gro -p lnpn-dipc-chol.top -t nvt -o npt.tpr -maxwarn 1"}

In [ ]:
!{"gmx mdrun -v -deffnm npt"}

## Generate production run .tpr file 

In [ ]:
!{"gmx grompp -f martini_md.mdp -n index.ndx -c npt.gro -r npt.gro -p lnpn-dipc-chol.top -o md.tpr -maxwarn 1"}

## Pass the file to the GPU 

In [ ]:
!{" sshpass -p \"shivrekhi\" rsync -zarvh md.tpr shivrekhi@128.2.54.30:~/GMXworkspace/LNPN-freestate"}